# Introduction


In this notebook we demostrate how to finetune **Llama3** (8B) model from Meta using **QLoRA** and **SFTTrainer** from **tlr**.

## What is Llama3?

Llama3 is the latest release of open-source LLMs from Meta, with features pretrained and instruction-fine-tuned language models with 8B and 70B parameters.

## What is LoRA?

LoRA stands for Low-Rank Adaptation. It is a method used to fine-tune large language models (LLMs) by freezing the weights of the LLM and injecting trainable rank-decomposition matrices. The number of trainable parameters during fine-tunning will decrease therefore considerably. According to LoRA paper, this number decreases 10,000 times, and the computational resources size decreases 3 times.


## What is QLoRA?

QLoRA builds on LoRA by incorporating quantization techniques to further reduce memory usage while maintaining, or even enhancing, model performance. With QLoRA it is possible to finetune a 70B parameter model that requires 36 GPUs with only 2!


## What is PEFT?

Parameter-efficient Fine-tuning (PEFT) is a technique used in Natural Language Processing (NLP) to improve the performance of pre-trained language models on specific downstream tasks. It involves reusing the pre-trained model’s parameters and fine-tuning them on a smaller dataset, which saves computational resources and time compared to training the entire model from scratch. PEFT achieves this efficiency by freezing some of the layers of the pre-trained model and only fine-tuning the last few layers that are specific to the downstream task.

## What is SFTTrainer?

SFT in SFTTrainer stands for supervised fine-tuning. The **trl** (Transformer Reinforcement Learning) library from HuggingFace provides a simple API to fine-tune models using SFTTrainer.

## What is UltraChat200k?  

UltraChat-200k is an invaluable resource for natural language understanding, generation and dialog system research. With 1.4 million dialogues spanning a variety of topics, this parquet-formatted dataset offers researchers four distinct formats to aid in their studies: test_sft, train_sft, train_gen and test_gen. More details [here](https://www.kaggle.com/datasets/thedevastator/ultrachat-200k-nlp-dataset).

## Inspiration

For this notebook, I took inspiration from several sources:
* [Efficiently fine-tune Llama 3 with PyTorch FSDP and Q-Lora](https://www.philschmid.de/fsdp-qlora-llama3)  
* [Fine-tuning LLMs using LoRA](https://medium.com/@rajatsharma_33357/fine-tuning-llama-using-lora-fb3f48a557d5)  
* [Fine-tuning Llama-3–8B-Instruct QLORA using low cost resources](https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04)  
* [Llama2 Fine-Tuning with Low-Rank Adaptations (LoRA) on Gaudi 2 Processors](https://eduand-alvarez.medium.com/llama2-fine-tuning-with-low-rank-adaptations-lora-on-gaudi-2-processors-52cf1ee6ce11)  

# Install and import libraries

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# wandb_key = user_secrets.get_secret("wandb_api")
# import wandb
# ! wandb login $wandb_key

In [ ]:
import os
import torch
from time import time
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    Trainer
)
from trl import SFTTrainer,setup_chat_format
import numpy as np

In [ ]:
class CFG:
    NUM_EPOCHS = 1
    BATCH_SIZE = 16
    DROPOUT = 0.05 
    MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
    
    SEED = 2024 
    MAX_LENGTH = 1024 
    NUM_WARMUP_STEPS = 128
    LR_MAX = 5e-5 
    NUM_LABELS = 3 
    LORA_RANK = 4
    LORA_ALPHA = 8
    LORA_MODULES = ['o_proj', 'v_proj']

# Prepare Dataset

In [ ]:
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)

# Drop 'Null' for training
indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index
train.drop(indexes, inplace=True)
train.reset_index(inplace=True, drop=True)
train.loc[:, 'label'] = np.argmax(train[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)

print(f"Total {len(indexes)} Null response rows dropped")
print('Total train samples: ', len(train))

train['text'] = 'User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']
print(train['text'][4])

# Initialize model


The model used is:

* **Model**: Llama3  
* **Framework**: Transformers   
* **Size**: 8B   
* **Type**: 8b-chat-hf (hf stands for HuggingFace). 
* **Version**: V1  

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True, 
    bnb_4bit_compute_dtype = torch.bfloat16 
)

model_name = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=4,
    device_map='auto'
)

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

lora_config = LoraConfig(
    r = 16, 
    lora_alpha = 8,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, 
    bias = 'none',
    task_type = 'SEQ_CLS'
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
from datasets import DatasetDict, Dataset

def data_preprocesing(row):
    return tokenizer(row['text'], padding='max_length', truncation=True, max_length=CFG.MAX_LENGTH, return_tensors='np')

dataset = Dataset.from_pandas(train)
tokenized_data = dataset.map(data_preprocesing, batched=True, remove_columns=['text'])
tokenized_data.set_format("torch")

In [ ]:
import gc

del train
gc.collect()

In [ ]:
from transformers import DataCollatorWithPadding

collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(evaluations):
    predictions, labels = evaluations
    predictions = np.argmax(predictions, axis=1)
    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),
    'accuracy':accuracy_score(predictions,labels)}

In [ ]:
import torch.nn.functional as F

class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
#         print(inputs)
        labels = inputs.pop("labels").long()

        outputs = model(**inputs)

        logits = outputs.get('logits')

        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir = 'sentiment_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 1,
    logging_steps=1,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    report_to="none"
)

In [ ]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data,
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
#     class_weights=class_weights,
)

train_result = trainer.train()